In [1]:
import pandas as pd
import datetime
import win32com.client
import requests
from bs4 import BeautifulSoup as bs
import time
import json

In [74]:
import pandas as pd
import datetime
import win32com.client
import requests
from bs4 import BeautifulSoup as bs
import time
import json
start_time = time.time()

def get_df_cur():
    url = 'https://www.investing.com/stock-screener/Service/SearchStocks'
    header = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-Site': 'same-origin',
            'X-Requested-With': 'XMLHttpRequest'
    }
    payload = {
        'country[]': '11',
        'sector': '7,5,12,3,8,9,1,6,2,4,10,11',
        'industry': '81,56,59,41,68,67,88,51,72,47,12,8,50,2,71,9,69,45,46,13,94,102,95,58,100,101,87,31,6,38,79,30,77,28,5,60,18,26,44,35,53,48,49,55,78,7,86,10,1,34,3,11,62,16,24,20,54,33,83,29,76,37,90,85,82,22,14,17,19,43,89,96,57,84,93,27,74,97,4,73,36,42,98,65,70,40,99,39,92,75,66,63,21,25,64,61,32,91,52,23,15,80',
        'equityType': 'ORD,DRC,Preferred,Unit,ClosedEnd,REIT,ELKS,OpenEnd,Right,ParticipationShare,CapitalSecurity,PerpetualCapitalSecurity,GuaranteeCertificate,IGC,Warrant,SeniorNote,Debenture,ETF,ADR,ETC,ETN',
        'pn': '1',
        'order[col]': 'eq_market_cap',
        'order[dir]': 'd',
    }
    df_curr = pd.DataFrame()
    print('current_id 추출 시작')
    for i in range(1,49): # 2373 / 50 = 47.46        
        print('남은 개수 : {}개'.format(49 - i))
        payload['pn'] = i
        r = requests.post(url, data = payload, headers = header)        
        dat = pd.DataFrame(json.loads(r.text)['hits'])[['pair_ID','stock_symbol']]
        df_curr = pd.concat([df_curr,dat])
    
    df_curr.columns = ['curr_id', 'num_code']
    df_curr.to_pickle('df_curr.pickle')
    return df_curr


def get_raw_data(short_code, st_date, end_date): #'20200128' 방식으로 date 입력
    num_code = short_code[1:]
    df_curr = pd.read_pickle('df_curr.pickle')
    curr_id = str(df_curr[df_curr['num_code'] == num_code].iloc[0,0])
    header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'X-Requested-With': 'XMLHttpRequest'
    }
    url = 'https://www.investing.com/instruments/HistoricalDataAjax'
    payload = {
        'interval_sec': 'Daily',
        'sort_col': 'date',
        'sort_ord': 'DESC',
        'action': 'historical_data'
    }

    def tr_date(date):
        year, month, day = date[:4], date[4:6], date[6:]
        tr_date = month + '/' + day + '/' + year
        return tr_date
    
    payload['curr_id'] = curr_id
    payload['header'] = num_code + ' ' + 'Historical Data'
    payload['st_date'] = tr_date(st_date)
    payload['end_date'] = tr_date(end_date)

    r1 = requests.post(url, data=payload, headers=header)
    tables = bs(r1.text, 'lxml').findAll('table')
    raw_data = pd.read_html(str(tables[0]))[0]
    return raw_data


def preprocess(short_code, raw_data):
    # columns 정리
    raw_data.drop(['Open', 'High', 'Low', 'Vol.', 'Change %'], axis='columns', inplace=True)
    raw_data.rename(columns={'Date': 'date', 'Price': 'adj'}, inplace=True)
    raw_data['short_code'] = short_code

    def tr_date_1(date_1):
        tr_month = {
            'Jan': 1,
            'Feb': 2,
            'Mar': 3,
            'Apr': 4,
            'May': 5,
            'Jun': 6,
            'Jul': 7,
            'Aug': 8,
            'Sep': 9,
            'Oct': 10,
            'Nov': 11,
            'Dec': 12
        }
        year, month, day = int(date_1[-4:]), tr_month[date_1[:3]], int(date_1[4:6])
        tr_date_var = datetime.datetime(year, month, day)
        return tr_date_var

    raw_data['date'] = raw_data['date'].apply(lambda x: tr_date_1(x))
    fined_data = raw_data
    return fined_data


def get_adj_stock(short_code, st_date, end_date):
    raw_data = get_raw_data(short_code, st_date, end_date)
    fined_data = preprocess(short_code, raw_data)
    return fined_data


def updater(adj_total):
    yst_day = datetime.datetime.today() - datetime.timedelta(days = 1)
    yst_day = yst_day.strftime('%Y%m%d')
    latest_day = (adj_total['date'].iloc[0] + datetime.timedelta(days = 1))
    latest_day = latest_day.strftime('%Y%m%d')

    short_code_list = adj_total['short_code'].unique().copy()
    if any(adj_total['date'].isin([yst_day])) == False:
        updat_list = []
        for idx,short_code in enumerate(short_code_list):
            print('남은 개수 : {0}, {1} 업데이트 중....'.format(len(short_code_list)-idx, short_code))        
            com = adj_total[adj_total['short_code'] == short_code].copy()
            try:
                new_data = get_adj_stock(short_code, latest_day, yst_day)
            except Exception as e:
                print(e)
            updat_list.append(new_data)        
            print('{0} 종목에 대해 {1}부터 {2}까지 업데이트 됨'.format(short_code, latest_day, yst_day))            

        return pd.concat(updat_list)

    else:
        statement = '현재 최신버전입니다.'.format('adj_total')
        print(statement)
        return True

def updater_merge():
    update = pd.read_pickle('update.pickle')
    adj_total = pd.read_pickle('adj_total.pickle')
    short_code_list = adj_total['short_code'].unique().copy()
    new_list = []
    for idx, short_code in enumerate(short_code_list):
        print(short_code)
        com_df = adj_total[adj_total['short_code'] == short_code].copy()
        update_df = update[update['short_code'] == short_code].copy()
        new_com_df = pd.concat([update_df,com_df])
        new_com_df.reset_index(drop = True, inplace = True)
        new_list.append(new_com_df)
        print('남은 개수 : {}개'.format(len(short_code_list) - idx)

    adj_total_update = pd.concat(new_list)
    return adj_total_update
    
    
    
if __name__ == '__main__':
    all_total = pd.read_pickle('./all_total.pickle')
    adj_list = []
    codes = all_total['short_code'].unique()
    for idx, short_code in enumerate(codes):
        remain = len(codes) -idx
        try:
            print(short_code, '남은 개수 : {}개'.format(remain))
            adj_list.append(get_adj_stock(short_code))
        except Exception as e:
            print(e)    
    """
    try:
        adj_total = pd.concat(adj_list)
        new_all_total = pd.merge(all_total, adj_total, on = ['date', 'short_code'])
    except Exception as e:
             print(e)
    adj_total.to_pickle('adj_total.pickle')
    """
    
    end_time = time.time()
    print('소요시간 : {}초'.format(end_time - start_time))

A238490 업데이트 중....


ValueError: invalid literal for int() with base 10: 'ound'

In [139]:
yst_day = datetime.datetime.today() - datetime.timedelta(days = 1)
yst_day = yst_day.strftime('%Y%m%d')
latest_day = (adj_total['date'].iloc[0] + datetime.timedelta(days = 1))
latest_day = latest_day.strftime('%Y%m%d')

short_code_list = adj_total['short_code'].unique().copy()
if any(adj_total['date'].isin([yst_day])) == False:
    updat_list = []
    for idx,short_code in enumerate(short_code_list):
        print('남은 개수 : {0}, {1} 업데이트 중....'.format(len(short_code_list)-idx, short_code))        
        com = adj_total[adj_total['short_code'] == short_code].copy()
        try:
            new_data = get_adj_stock(short_code, latest_day, yst_day)
        except Exception as e:
            print(e)
        updat_list.append(new_data)        
        print('{0} 종목에 대해 {1}부터 {2}까지 업데이트 됨'.format(short_code, latest_day, yst_day))
        # com_1 = pd.concat([new_data, com]).reset_index(drop = True, inplace = True)        
        # x = pd.concat([com_1, x]).drop_duplicates()
        
    # return pd.concat(new_data)

else:
    statement = '현재 최신버전입니다.'.format(df.name)
    print(statement)
    # return True              

남은 개수 : 2326, A060310 업데이트 중....
A060310 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2325, A095570 업데이트 중....
A095570 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2324, A006840 업데이트 중....
A006840 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2323, A054620 업데이트 중....
A054620 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2322, A265520 업데이트 중....
A265520 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2321, A211270 업데이트 중....
A211270 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2320, A027410 업데이트 중....
A027410 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2319, A282330 업데이트 중....
A282330 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2318, A138930 업데이트 중....
A138930 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2317, A001460 업데이트 중....
A001460 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2316, A001040 업데이트 중....
A001040 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2315, A079160 업데이트 중....
A079160 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2314, A035760 업데이트 중....
A035760 종목에 대해 20200121부터 20200127까지 업데이트 됨

A011070 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2221, A066570 업데이트 중....
A066570 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2220, A108670 업데이트 중....
A108670 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2219, A037560 업데이트 중....
A037560 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2218, A051910 업데이트 중....
A051910 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2217, A079550 업데이트 중....
A079550 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2216, A183350 업데이트 중....
invalid literal for int() with base 10: 'ound'
A183350 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2215, A006260 업데이트 중....
A006260 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2214, A000680 업데이트 중....
A000680 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2213, A010120 업데이트 중....
A010120 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2212, A229640 업데이트 중....
A229640 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2211, A023150 업데이트 중....
A023150 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2210, A219550 업데이트 중....
A219550 종목에 대해 20200121부터 202

A009450 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2116, A267290 업데이트 중....
A267290 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2115, A012320 업데이트 중....
A012320 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2114, A011040 업데이트 중....
A011040 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2113, A000050 업데이트 중....
A000050 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2112, A214390 업데이트 중....
A214390 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2111, A012610 업데이트 중....
A012610 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2110, A009140 업데이트 중....
A009140 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2109, A024910 업데이트 중....
A024910 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2108, A013580 업데이트 중....
A013580 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2107, A012200 업데이트 중....
A012200 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2106, A004200 업데이트 중....
A004200 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2105, A002140 업데이트 중....
A002140 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2104, A198440 업데이트 중....

invalid literal for int() with base 10: 'ound'
A153460 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2009, A007390 업데이트 중....
A007390 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2008, A086220 업데이트 중....
A086220 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2007, A033640 업데이트 중....
A033640 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2006, A005720 업데이트 중....
A005720 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2005, A002350 업데이트 중....
A002350 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2004, A089140 업데이트 중....
A089140 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2003, A065170 업데이트 중....
A065170 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2002, A003580 업데이트 중....
A003580 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2001, A137940 업데이트 중....
A137940 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2000, A041140 업데이트 중....
A041140 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 1999, A217270 업데이트 중....
A217270 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 1998, A225570 업데이트 중....
A225570 종목에 대해 20200121부터 202

A290120 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 1904, A000300 업데이트 중....
A000300 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 1903, A120240 업데이트 중....
A120240 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 1902, A003310 업데이트 중....
A003310 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 1901, A114920 업데이트 중....
A114920 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 1900, A078600 업데이트 중....
A078600 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 1899, A012800 업데이트 중....
A012800 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 1898, A015230 업데이트 중....
A015230 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 1897, A096350 업데이트 중....
A096350 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 1896, A140520 업데이트 중....
A140520 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 1895, A131220 업데이트 중....
A131220 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 1894, A010170 업데이트 중....
A010170 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 1893, A054670 업데이트 중....
A054670 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 1892, A001070 업데이트 중....

...

A264900 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 436, A005740 업데이트 중....
A005740 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 435, A215570 업데이트 중....
A215570 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 434, A043590 업데이트 중....
invalid literal for int() with base 10: 'ound'
A043590 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 433, A114120 업데이트 중....
A114120 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 432, A110790 업데이트 중....
A110790 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 431, A083790 업데이트 중....
A083790 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 430, A900250 업데이트 중....
A900250 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 429, A045520 업데이트 중....
A045520 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 428, A214150 업데이트 중....
A214150 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 427, A237880 업데이트 중....
A237880 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 426, A139670 업데이트 중....
A139670 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 425, A020120 업데이트 중....
A020120 종목에 대해 20200121부터 20200127까지 업데이트

A054300 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 330, A068050 업데이트 중....
A068050 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 329, A028670 업데이트 중....
A028670 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 328, A222110 업데이트 중....
A222110 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 327, A010820 업데이트 중....
A010820 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 326, A016800 업데이트 중....
A016800 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 325, A263750 업데이트 중....
A263750 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 324, A251970 업데이트 중....
A251970 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 323, A001020 업데이트 중....
A001020 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 322, A327610 업데이트 중....
A327610 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 321, A087010 업데이트 중....
A087010 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 320, A090080 업데이트 중....
A090080 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 319, A043370 업데이트 중....
A043370 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 318, A010770 업데이트 중....
A010770 종목에 

A039340 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 226, A005430 업데이트 중....
A005430 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 225, A071050 업데이트 중....
A071050 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 224, A034950 업데이트 중....
A034950 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 223, A010040 업데이트 중....
A010040 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 222, A025540 업데이트 중....
A025540 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 221, A222980 업데이트 중....
A222980 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 220, A318000 업데이트 중....
A318000 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 219, A256840 업데이트 중....
A256840 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 218, A004090 업데이트 중....
A004090 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 217, A025550 업데이트 중....
A025550 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 216, A002200 업데이트 중....
A002200 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 215, A002960 업데이트 중....
A002960 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 214, A023890 업데이트 중....
A023890 종목에 

A003480 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 118, A180640 업데이트 중....
A180640 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 117, A005110 업데이트 중....
A005110 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 116, A079170 업데이트 중....
A079170 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 115, A009460 업데이트 중....
A009460 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 114, A086960 업데이트 중....
A086960 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 113, A054920 업데이트 중....
A054920 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 112, A002680 업데이트 중....
A002680 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 111, A066110 업데이트 중....
A066110 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 110, A000880 업데이트 중....
A000880 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 109, A027390 업데이트 중....
A027390 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 108, A088350 업데이트 중....
A088350 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 107, A000370 업데이트 중....
A000370 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 106, A009830 업데이트 중....
A009830 종목에 

A005010 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 11, A243070 업데이트 중....
A243070 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 10, A084110 업데이트 중....
A084110 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 9, A145020 업데이트 중....
A145020 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 8, A069260 업데이트 중....
A069260 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 7, A024060 업데이트 중....
A024060 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 6, A010240 업데이트 중....
A010240 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 5, A189980 업데이트 중....
A189980 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 4, A000540 업데이트 중....
A000540 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 3, A003280 업데이트 중....
A003280 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 2, A037440 업데이트 중....
A037440 종목에 대해 20200121부터 20200127까지 업데이트 됨
남은 개수 : 1, A238490 업데이트 중....
A238490 종목에 대해 20200121부터 20200127까지 업데이트 됨


In [2]:
update = pd.read_pickle('update.pickle')
adj_total = pd.read_pickle('adj_total.pickle')

In [4]:
update.head()

,date,adj,short_code
0,2020-01-23,2860.0,A060310
1,2020-01-22,2880.0,A060310
2,2020-01-21,2890.0,A060310
0,2020-01-23,4960.0,A095570
1,2020-01-22,4855.0,A095570


In [5]:
adj_total.head()

,date,adj,short_code
0,2020-01-20,2880.0,A060310
1,2020-01-17,2930.0,A060310
2,2020-01-16,2905.0,A060310
3,2020-01-15,2800.0,A060310
4,2020-01-14,2805.0,A060310


In [8]:
start_time = time.time()
short_code_list = adj_total['short_code'].unique().copy()
new_list = []
for idx, short_code in enumerate(short_code_list):
    print(short_code)
    com_df = adj_total[adj_total['short_code'] == short_code].copy()
    update_df = update[update['short_code'] == short_code].copy()
    new_com_df = pd.concat([update_df,com_df])
    new_com_df.reset_index(drop = True, inplace = True)
    new_list.append(new_com_df)
    print('남은 개수 : {}개'.format(len(short_code_list) - idx)
    
adj_total_update = pd.concat(new_list)
end_time = time.time()
total_time = (start_time - end_time)/60
print('총 경과시간 : {}'.format(total_time))

A060310
A095570
A006840
A054620
A265520
A211270
A027410
A282330
A138930
A001460
A001040
A079160
A035760
A000120
A011150
A097950
A051500
A058820
A023460
A065770
A083660
A000590
A012030
A322780
A016610
A005830
A000990
A139130
A068790
A004840
A241520
A155660
A069730
A017940
A245620
A037370
A079190
A007700
A130500
A114090
A900290
A078930
A083450
A006360
A001250
A007070
A028150
A045890
A078150
A012630
A039570
A089470
A294870
A036640
A082740
A299170
A335890
A335870
A276920
A095340
A099520
A175330
A950170
A234080
A067290
A001060
A096760
A035900
A024840
A105560
A024120
A021320
A036670
A009440
A112190
A119650
A044180
A092220
A151860
A046440
A035600
A001390
A060720
A001940
A083470
A122450
A052900
A058400
A114450
A025000
A092230
A000040
A044450
A030210
A036030
A058850
A058860
A060370
A093050
A003550
A034220
A001120
A051900
A032640
A011070
A066570
A108670
A037560
A051910
A079550
A183350
A006260
A000680
A010120
A229640
A023150
A219550
A035420
A160550
A053290
A104200
A060250
A005940
A338100
A034310


A031430
A031440
A006880
A056000
A002800
A323280
A005800
A001720
A009270
A017000
A002700
A012790
A138070
A226330
A019170
A002870
A005450
A293940
A277480
A323230
A333050
A055550
A001770
A056700
A187270
A004080
A243840
A108320
A222800
A036710
A160980
A217330
A102280
A003410
A003620
A010280
A004770
A217320
A208640
A222420
A037760
A136510
A099320
A049960
A050890
A066790
A222080
A004920
A236030
A103660
A245450
A264660
A297090
A112610
A115530
A225330
A115480
A189330
A096530
A900120
A101240
A047920
A189540
A060590
A036170
A260930
A013990
A123860
A025980
A008700
A058220
A125210
A090430
A002790
A052710
A074430
A092040
A083930
A149950
A036010
A002030
A183190
A002310
A050860
A246720
A067390
A127710
A267850
A020560
A154030
A227610
A143160
A054800
A122900
A099190
A289010
A214430
A040910
A068940
A052860
A010780
A069920
A038880
A123010
A003560
A307180
A101390
A226350
A078860
A114810
A031310
A185490
A059100
A149010
A175250
A052460
A119830
A124500
A084850
A223220
A027360
A033660
A032080
A013310
A059120


A002230
A242350
A137400
A128660
A006140
A062970
A087600
A057880
A033180
A064800
A161580
A270520
A284620
A320000
A332710
A261200
A086790
A099340
A099350
A067310
A307160
A166090
A293480
A039130
A136480
A003380
A172580
A153360
A159650
A149980
A013030
A126700
A200470
A106080
A071090
A317240
A340120
A221840
A106190
A019490
A000080
A000140
A066130
A152550
A004590
A036460
A039340
A005430
A071050
A034950
A010040
A025540
A222980
A318000
A256840
A004090
A025550
A002200
A002960
A023890
A017890
A080720
A123890
A015760
A063570
A041460
A006200
A101680
A039740
A053300
A025770
A271740
A281410
A291210
A310870
A002300
A009540
A023350
A025890
A000970
A104700
A017960
A023760
A054040
A050540
A161890
A024720
A021650
A161390
A053590
A000240
A034830
A007280
A168490
A037230
A010100
A047810
A123690
A003350
A030520
A052600
A011500
A002390
A256150
A014790
A092460
A060980
A222810
A053690
A042700
A008930
A128940
A047080
A009240
A020000
A003680
A105630
A069640
A016450
A009180
A070300
A221610
A070590
A213500
A014680


In [11]:
adj_total_update.to_pickle('adj_total20200128.pickle')

In [30]:
adj_total = pd.concat(adj_list)

In [32]:
adj_total.to_pickle('adj_total.pickle')

In [11]:
datetime(2020,1,20)

TypeError: 'module' object is not callable

In [45]:
new_all_total = pd.merge(all_total, adj_total, on = ['date', 'short_code'])

In [47]:
new_all_total.to_pickle('new_all_total.pickle')

In [48]:
new_all_total

,short_code,codeName,date,close_price,up_down,diff,high_price,low_price,open_price,trd_vol_won,trd_vol_stocks,listed_shares,market_cap,adj
0,A060310,3S,2020-01-20,2880,2,50,2965,2860,2965,373252815,128911,44772143,128944,2880
1,A060310,3S,2020-01-20,2880,2,50,2965,2860,2965,373252815,128911,44772143,128944,2930
2,A060310,3S,2020-01-20,2880,2,50,2965,2860,2965,373252815,128911,44772143,128944,2905
3,A060310,3S,2020-01-20,2880,2,50,2965,2860,2965,373252815,128911,44772143,128944,2800
4,A060310,3S,2020-01-20,2880,2,50,2965,2860,2965,373252815,128911,44772143,128944,2805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099811,A238490,힘스,2020-01-20,25150,2,350,25600,25100,25600,2126830450,84298,5656118,142251,24050
1099812,A238490,힘스,2020-01-20,25150,2,350,25600,25100,25600,2126830450,84298,5656118,142251,23050
1099813,A238490,힘스,2020-01-20,25150,2,350,25600,25100,25600,2126830450,84298,5656118,142251,22950
1099814,A238490,힘스,2020-01-20,25150,2,350,25600,25100,25600,2126830450,84298,5656118,142251,23200


In [37]:
adj_total['date'].iloc[0]

datetime.date(2020, 1, 20)

In [38]:
x = datetime.date(2020,1,20)

In [40]:
type(x.year)

int

In [26]:
all_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1084623 entries, 0 to 1084622
Data columns (total 13 columns):
short_code        1084623 non-null object
codeName          1084623 non-null object
date              1084623 non-null datetime64[ns]
close_price       1084623 non-null int64
up_down           1084623 non-null int64
diff              1084623 non-null int64
high_price        1084623 non-null int64
low_price         1084623 non-null int64
open_price        1084623 non-null int64
trd_vol_won       1084623 non-null int64
trd_vol_stocks    1084623 non-null int64
listed_shares     1084623 non-null int64
market_cap        1084623 non-null int64
dtypes: datetime64[ns](1), int64(10), object(2)
memory usage: 107.6+ MB


In [24]:
pd.merge(all_total, adj_total, on = ['date', 'short_code'])

ValueError: You are trying to merge on datetime64[ns] and object columns. If you wish to proceed you should use pd.concat

In [11]:
adj_total

,date,adj,short_code
0,2020-01-20,62400,A060310
1,2020-01-19,61300,A060310
2,2020-01-17,61300,A060310
3,2020-01-16,60700,A060310
4,2020-01-15,59000,A060310
...,...,...,...
547,2018-01-26,50780,A238490
548,2018-01-25,50260,A238490
549,2018-01-24,49340,A238490
550,2018-01-23,49160,A238490


In [28]:
all_total

,short_code,codeName,date,close_price,up_down,diff,high_price,low_price,open_price,trd_vol_won,trd_vol_stocks,listed_shares,market_cap
0,A060310,3S,2020-01-20,2880,2,50,2965,2860,2965,373252815,128911,44772143,128944
1,A060310,3S,2020-01-17,2930,1,25,2985,2885,2900,310744825,105773,44772143,131182
2,A060310,3S,2020-01-16,2905,1,105,2950,2800,2825,896431915,308911,44772143,130063
3,A060310,3S,2020-01-15,2800,2,5,2825,2780,2805,227249740,81013,44772143,125362
4,A060310,3S,2020-01-14,2805,2,10,2845,2795,2830,236135665,83827,44772143,125586
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084618,A238490,힘스,2018-01-26,24050,1,1000,24250,23000,23100,3577092600,150190,5258150,126459
1084619,A238490,힘스,2018-01-25,23050,1,100,23150,22850,23000,931717200,40547,5258150,121200
1084620,A238490,힘스,2018-01-24,22950,2,250,23400,22850,23400,1171441300,51011,5258150,120675
1084621,A238490,힘스,2018-01-23,23200,1,300,23250,22500,22950,1412728550,61729,5258150,121989


In [20]:
adj_total1 = adj_total.copy()

In [21]:
adj_total1['date'] = adj_total1['date'].apply(lambda x : str(x))

In [24]:
new_all_total = pd.merge(all_total, adj_total1, on = ['date'])

ValueError: negative dimensions are not allowed

In [30]:
adj_total1.iloc[0,0] == all_total.iloc[0,2]

True

In [ ]:
print(all_total['date'].dtype)
print(adj_total['date'].dtype)
print(all_total['date'][0] == adj_total['date'][0])

In [ ]:
new_all_total = pd.merge(all_total, adj_total, on = ['date', 'short_code'])

In [ ]:
new_all_total.to_pickle('all_total.pickle')

In [9]:
import requests
import pandas as pd
import json

In [35]:
url = 'https://www.investing.com/stock-screener/Service/SearchStocks'
header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'X-Requested-With': 'XMLHttpRequest'
}
payload = {
    'country[]': '11',
    'sector': '7,5,12,3,8,9,1,6,2,4,10,11',
    'industry': '81,56,59,41,68,67,88,51,72,47,12,8,50,2,71,9,69,45,46,13,94,102,95,58,100,101,87,31,6,38,79,30,77,28,5,60,18,26,44,35,53,48,49,55,78,7,86,10,1,34,3,11,62,16,24,20,54,33,83,29,76,37,90,85,82,22,14,17,19,43,89,96,57,84,93,27,74,97,4,73,36,42,98,65,70,40,99,39,92,75,66,63,21,25,64,61,32,91,52,23,15,80',
    'equityType': 'ORD,DRC,Preferred,Unit,ClosedEnd,REIT,ELKS,OpenEnd,Right,ParticipationShare,CapitalSecurity,PerpetualCapitalSecurity,GuaranteeCertificate,IGC,Warrant,SeniorNote,Debenture,ETF,ADR,ETC,ETN',
    'pn': '1',
    'order[col]': 'eq_market_cap',
    'order[dir]': 'd',
}


In [36]:
tmp = []
for i in range(1,49): # 2373 / 50 = 47.46
    print(i)
    payload['pn'] = i
    r = requests.post(url, data = payload, headers = header)
    dat = pd.DataFrame(json.loads(r.text)['hits'])
    tmp.append(dat)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47


In [46]:
tmp2 = []
for df in tmp:
    tmp2.append(df[['pair_ID','stock_symbol']])
df_curr = pd.concat(tmp2)
df_curr.columns = ['curr_id', 'num_code']

In [50]:
df_curr = pd.concat(tmp2)
df_curr.columns = ['curr_id', 'num_code']
df_curr

,curr_id,num_code
0,951042,005935
1,43433,005930
2,43430,000660
3,43399,005380
4,992636,005385
...,...,...
18,1155908,317860
19,1155951,322510
20,1155952,124560
21,1156084,288330


In [55]:
num_code = '005930'
str(df_curr[df_curr['num_code'] == num_code].iloc[0,0])

'43433'

In [56]:
url = 'https://www.investing.com/stock-screener/Service/SearchStocks'
header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'X-Requested-With': 'XMLHttpRequest'
}
payload = {
    'country[]': '11',
    'sector': '7,5,12,3,8,9,1,6,2,4,10,11',
    'industry': '81,56,59,41,68,67,88,51,72,47,12,8,50,2,71,9,69,45,46,13,94,102,95,58,100,101,87,31,6,38,79,30,77,28,5,60,18,26,44,35,53,48,49,55,78,7,86,10,1,34,3,11,62,16,24,20,54,33,83,29,76,37,90,85,82,22,14,17,19,43,89,96,57,84,93,27,74,97,4,73,36,42,98,65,70,40,99,39,92,75,66,63,21,25,64,61,32,91,52,23,15,80',
    'equityType': 'ORD,DRC,Preferred,Unit,ClosedEnd,REIT,ELKS,OpenEnd,Right,ParticipationShare,CapitalSecurity,PerpetualCapitalSecurity,GuaranteeCertificate,IGC,Warrant,SeniorNote,Debenture,ETF,ADR,ETC,ETN',
    'pn': '1',
    'order[col]': 'eq_market_cap',
    'order[dir]': 'd',
}
df_curr = pd.DataFrame()
for i in range(1,49): # 2373 / 50 = 47.46
    print(i)
    payload['pn'] = i
    r = requests.post(url, data = payload, headers = header)        
    dat = pd.DataFrame(json.loads(r.text)['hits'])[['pair_ID','stock_symbol']]
    df_curr = pd.concat([df_curr,dat])

df_curr.columns = ['curr_id', 'num_code']
df_curr

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48


,curr_id,num_code
0,951042,005935
1,43433,005930
2,43430,000660
3,43399,005380
4,992636,005385
...,...,...
18,1155908,317860
19,1155951,322510
20,1155952,124560
21,1156084,288330


In [63]:
pd.DataFrame({'date' : [1,2,3,4,5,6], 'price' : [15,23,54,26,82,36]})

,date,price
0,1,15
1,2,23
2,3,54
3,4,26
4,5,82
5,6,36


In [4]:
import time
def get_df_cur():
    start_time = time.time()
    url = 'https://www.investing.com/stock-screener/Service/SearchStocks'
    header = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-Site': 'same-origin',
            'X-Requested-With': 'XMLHttpRequest'
    }
    payload = {
        'country[]': '11',
        'sector': '7,5,12,3,8,9,1,6,2,4,10,11',
        'industry': '81,56,59,41,68,67,88,51,72,47,12,8,50,2,71,9,69,45,46,13,94,102,95,58,100,101,87,31,6,38,79,30,77,28,5,60,18,26,44,35,53,48,49,55,78,7,86,10,1,34,3,11,62,16,24,20,54,33,83,29,76,37,90,85,82,22,14,17,19,43,89,96,57,84,93,27,74,97,4,73,36,42,98,65,70,40,99,39,92,75,66,63,21,25,64,61,32,91,52,23,15,80',
        'equityType': 'ORD,DRC,Preferred,Unit,ClosedEnd,REIT,ELKS,OpenEnd,Right,ParticipationShare,CapitalSecurity,PerpetualCapitalSecurity,GuaranteeCertificate,IGC,Warrant,SeniorNote,Debenture,ETF,ADR,ETC,ETN',
        'pn': '1',
        'order[col]': 'eq_market_cap',
        'order[dir]': 'd',
    }
    df_curr = pd.DataFrame()
    for i in range(1,49): # 2373 / 50 = 47.46
        print(i)
        payload['pn'] = i
        r = requests.post(url, data = payload, headers = header)        
        dat = pd.DataFrame(json.loads(r.text)['hits'])[['pair_ID','stock_symbol']]
        df_curr = pd.concat([df_curr,dat])
    
    df_curr.columns = ['curr_id', 'num_code']
    end_time = time.time()
    print(end_time - start_time)
    return df_curr

In [5]:
import multiprocessing

In [6]:
pool = multiprocessing.Pool(processes=2)

In [ ]:
pool.map(get_df_cur, [1,2])

In [ ]:
import pandas as pd
import datetime
import win32com.client
import requests
from bs4 import BeautifulSoup as bs
import time
import json
import multiprocessing


def get_df_cur():
    url = 'https://www.investing.com/stock-screener/Service/SearchStocks'
    header = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-Site': 'same-origin',
            'X-Requested-With': 'XMLHttpRequest'
    }
    payload = {
        'country[]': '11',
        'sector': '7,5,12,3,8,9,1,6,2,4,10,11',
        'industry': '81,56,59,41,68,67,88,51,72,47,12,8,50,2,71,9,69,45,46,13,94,102,95,58,100,101,87,31,6,38,79,30,77,28,5,60,18,26,44,35,53,48,49,55,78,7,86,10,1,34,3,11,62,16,24,20,54,33,83,29,76,37,90,85,82,22,14,17,19,43,89,96,57,84,93,27,74,97,4,73,36,42,98,65,70,40,99,39,92,75,66,63,21,25,64,61,32,91,52,23,15,80',
        'equityType': 'ORD,DRC,Preferred,Unit,ClosedEnd,REIT,ELKS,OpenEnd,Right,ParticipationShare,CapitalSecurity,PerpetualCapitalSecurity,GuaranteeCertificate,IGC,Warrant,SeniorNote,Debenture,ETF,ADR,ETC,ETN',
        'pn': '1',
        'order[col]': 'eq_market_cap',
        'order[dir]': 'd',
    }
    df_curr = pd.DataFrame()
    for i in range(1,49): # 2373 / 50 = 47.46
        print(i)
        payload['pn'] = i
        r = requests.post(url, data = payload, headers = header)        
        dat = pd.DataFrame(json.loads(r.text)['hits'])[['pair_ID','stock_symbol']]
        df_curr = pd.concat([df_curr,dat])
    
    df_curr.columns = ['curr_id', 'num_code']
    return df_curr


def get_raw_data(short_code, st_date, end_date):
    num_code = short_code[1:]
    # df_curr = get_df_cur()
    curr_id = str(df_curr[df_curr['num_code'] == num_code].iloc[0,0])
    header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'X-Requested-With': 'XMLHttpRequest'
    }
    url = 'https://www.investing.com/instruments/HistoricalDataAjax'
    payload = {
        'interval_sec': 'Daily',
        'sort_col': 'date',
        'sort_ord': 'DESC',
        'action': 'historical_data'
    }

    def tr_date(date):
        year, month, day = date[:4], date[4:6], date[6:]
        tr_date = month + '/' + day + '/' + year
        return tr_date
    
    payload['curr_id'] = curr_id
    payload['header'] = num_code + ' ' + 'Historical Data'
    payload['st_date'] = tr_date(st_date)
    payload['end_date'] = tr_date(end_date)

    r1 = requests.post(url, data=payload, headers=header)
    tables = bs(r1.text, 'lxml').findAll('table')
    raw_data = pd.read_html(str(tables[0]))[0]
    return raw_data


def preprocess(short_code, raw_data):
    # columns 정리
    raw_data.drop(['Open', 'High', 'Low', 'Vol.', 'Change %'], axis='columns', inplace=True)
    raw_data.rename(columns={'Date': 'date', 'Price': 'adj'}, inplace=True)
    raw_data['short_code'] = short_code

    def tr_date_1(date_1):
        tr_month = {
            'Jan': 1,
            'Feb': 2,
            'Mar': 3,
            'Apr': 4,
            'May': 5,
            'Jun': 6,
            'Jul': 7,
            'Aug': 8,
            'Sep': 9,
            'Oct': 10,
            'Nov': 11,
            'Dec': 12
        }
        year, month, day = int(date_1[-4:]), tr_month[date_1[:3]], int(date_1[4:6])
        tr_date_var = datetime.date(year, month, day)
        return tr_date_var

    raw_data['date'] = raw_data['date'].apply(lambda x: tr_date_1(x))
    fined_data = raw_data
    return fined_data


def get_adj_stock(short_code):
    raw_data = get_raw_data(short_code, '20180122', '20200120')
    fined_data = preprocess(short_code, raw_data)
    return fined_data

if __name__ == "__main__":  
    start_time = time.time()
    all_total = pd.read_pickle('./all_total.pickle')
    codes = all_total['short_code'].unique()

    pool = multiprocessing.Pool(8)
    adj_list = pool.map(get_adj_stock, codes)
    adj_total = pd.concat(adj_list)
    new_all_total = pd.merge(all_total, adj_total, on = ['date', 'short_code'])
   
    end_time = time.time()
    print('소요시간 : {}초'.format(end_time - start_time))

In [1]:
import pandas as pd
import datetime
import win32com.client
import requests
from bs4 import BeautifulSoup as bs
import time
import json
start_time = time.time()

def get_df_cur():
    url = 'https://www.investing.com/stock-screener/Service/SearchStocks'
    header = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-Site': 'same-origin',
            'X-Requested-With': 'XMLHttpRequest'
    }
    payload = {
        'country[]': '11',
        'sector': '7,5,12,3,8,9,1,6,2,4,10,11',
        'industry': '81,56,59,41,68,67,88,51,72,47,12,8,50,2,71,9,69,45,46,13,94,102,95,58,100,101,87,31,6,38,79,30,77,28,5,60,18,26,44,35,53,48,49,55,78,7,86,10,1,34,3,11,62,16,24,20,54,33,83,29,76,37,90,85,82,22,14,17,19,43,89,96,57,84,93,27,74,97,4,73,36,42,98,65,70,40,99,39,92,75,66,63,21,25,64,61,32,91,52,23,15,80',
        'equityType': 'ORD,DRC,Preferred,Unit,ClosedEnd,REIT,ELKS,OpenEnd,Right,ParticipationShare,CapitalSecurity,PerpetualCapitalSecurity,GuaranteeCertificate,IGC,Warrant,SeniorNote,Debenture,ETF,ADR,ETC,ETN',
        'pn': '1',
        'order[col]': 'eq_market_cap',
        'order[dir]': 'd',
    }
    df_curr = pd.DataFrame()
    print('current_id 추출 시작')
    for i in range(1,49): # 2373 / 50 = 47.46        
        print('남은 개수 : {}개'.format(49 - i))
        payload['pn'] = i
        r = requests.post(url, data = payload, headers = header)        
        dat = pd.DataFrame(json.loads(r.text)['hits'])[['pair_ID','stock_symbol']]
        df_curr = pd.concat([df_curr,dat])
    
    df_curr.columns = ['curr_id', 'num_code']
    df_curr.to_pickle('df_curr.pickle')
    return df_curr


def get_raw_data(short_code, st_date, end_date):
    num_code = short_code[1:]
    df_curr = pd.read_pickle('df_curr.pickle')
    curr_id = str(df_curr[df_curr['num_code'] == num_code].iloc[0,0])
    header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'X-Requested-With': 'XMLHttpRequest'
    }
    url = 'https://www.investing.com/instruments/HistoricalDataAjax'
    payload = {
        'interval_sec': 'Daily',
        'sort_col': 'date',
        'sort_ord': 'DESC',
        'action': 'historical_data'
    }

    def tr_date(date):
        year, month, day = date[:4], date[4:6], date[6:]
        tr_date = month + '/' + day + '/' + year
        return tr_date
    
    payload['curr_id'] = curr_id
    payload['header'] = num_code + ' ' + 'Historical Data'
    payload['st_date'] = tr_date(st_date)
    payload['end_date'] = tr_date(end_date)

    r1 = requests.post(url, data=payload, headers=header)
    tables = bs(r1.text, 'lxml').findAll('table')
    raw_data = pd.read_html(str(tables[0]))[0]
    return raw_data


def preprocess(short_code, raw_data):
    # columns 정리
    raw_data.drop(['Open', 'High', 'Low', 'Vol.', 'Change %'], axis='columns', inplace=True)
    raw_data.rename(columns={'Date': 'date', 'Price': 'adj'}, inplace=True)
    raw_data['short_code'] = short_code

    def tr_date_1(date_1):
        tr_month = {
            'Jan': 1,
            'Feb': 2,
            'Mar': 3,
            'Apr': 4,
            'May': 5,
            'Jun': 6,
            'Jul': 7,
            'Aug': 8,
            'Sep': 9,
            'Oct': 10,
            'Nov': 11,
            'Dec': 12
        }
        year, month, day = int(date_1[-4:]), tr_month[date_1[:3]], int(date_1[4:6])
        tr_date_var = datetime.datetime(year, month, day)
        return tr_date_var

    raw_data['date'] = raw_data['date'].apply(lambda x: tr_date_1(x))
    fined_data = raw_data
    return fined_data


def get_adj_stock(short_code):
    raw_data = get_raw_data(short_code, '20180122', '20200120')
    fined_data = preprocess(short_code, raw_data)
    return fined_data

In [2]:
get_adj_stock('A005930')

,date,adj,short_code
0,2020-01-20,62400,A005930
1,2020-01-19,61300,A005930
2,2020-01-17,61300,A005930
3,2020-01-16,60700,A005930
4,2020-01-15,59000,A005930
...,...,...,...
547,2018-01-26,50780,A005930
548,2018-01-25,50260,A005930
549,2018-01-24,49340,A005930
550,2018-01-23,49160,A005930
